# Getting Started with Mistral Tool Use & the Converse API

## Introduction
This notebook provides a guide on utilizing the Converse API for tool-use with the Mistral Large model. Our focus will be on defining tools, understanding how tool_choice impacts Mistral's output, and leveraging the Converse API to enhance the functionality of Mistral Large in various scenarios.

## Objectives
By the end of this notebook, you will:
- Understand how to define and configure tools for use with the Mistral-Large model.
- Learn the effects of different tool choices on Mistral's output.
- Gain practical experience with the Converse API, including tips and tricks to maximize the potential of Mistral-Large with tool-use.

## Steps for Tool Use

At a glance, there are four steps involved in function calling with the Mistral Large model:

1. **User: Specify Tools and Query**
   - Define the toolConfig json object, such as the `shinkansen_schedule` toolconfig that described the tool to Mistral-Large, and set up the query for the model.

2. **Model: Generate Tool Arguments if Applicable**
   - The model generates the necessary arguments for the specified tool based on the user query.

3. **User: Execute Function to Obtain Tool Results**
   - The user executes the function `shinkansen_schedule` with the generated arguments to obtain the tool results.

4. **Model: Generate Final Answer**
   - The model uses the tool results to generate the final answer to the user's query.


## Supported Models
Tool use is supported by Mistral Large & Mistral Small. 

### Mistral-Large
This model is designed to handle complex tasks and can be enhanced through tool use for more specific and actionable responses.

## Getting Started
Let's dive into practical examples to see how we can leverage the Converse API for tool use with the Mistral Large model. We'll begin by setting up our environment and configuring the necessary parameters.

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError

In [ ]:
modelId = 'mistral.mistral-large-2402-v1:0'
#modelId= 'mistral.mistral-small-2402-v1:0' #Mistral Small model ID
region = 'us-west-2'  

print(f'Using modelId: {modelId}')
print(f'Using region: {region}')

In [ ]:
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

## Basic Interaction with Mistral Large

Let's first check to see if Mistral-Large can answer a simple query without tool use.

In [ ]:
converse_api_params = {
    "modelId": modelId, 
    "messages": [{"role": "user", "content": [{"text": "What train departs Tokyo at 9:00"}]}],
    "inferenceConfig": {"temperature": 0.0, "maxTokens": 100}
}

response = bedrock_client.converse(**converse_api_params)

print(response['output']['message']['content'][0]['text'])

As expected, Mistral Large does not have access to real-time information and responds accordingly. We can enhance this interaction by using system roles and tool-use.

## Leveraging Tool Use

To provide more accurate and actionable responses, we can define and use tools with the Converse API.

### Defining the Shinkansen Schedule Tool

Here we define our tool for Mistral. It's important to write a clear description for Mistral in order the model to be able to know when to use the tool. Remember, Mistral does not actually take action by calling an api directly. The model simply decides which tool to use based on the user query and the description you provide for your tool. You need define the function that actually executes the action.

In [ ]:
# Define the tool configuration
toolConfig = {
    "tools": [
        {
            "toolSpec": {
                "name": "shinkansen_schedule",
                "description": "Fetches Shinkansen train schedule departure times for a specified station and time.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "station": {
                                "type": "string",
                                "description": "The station name."
                            },
                            "departure_time": {
                                "type": "string",
                                "description": "The departure time in HH:MM format."
                            }
                        },
                        "required": ["station", "departure_time"]
                    }
                }
            }
        }
    ]
}

Here, we define a mock function to simulate fetching train schedule data and configure it as a tool for Mistral Large.

In [ ]:
def shinkansen_schedule(station, departure_time):
    # This is a mock function to simulate fetching train schedule data
    schedule = {
        "Tokyo": {"09:00": "Hikari", "12:00": "Nozomi", "15:00": "Kodama"},
        "Osaka": {"10:00": "Nozomi", "13:00": "Hikari", "16:00": "Kodama"}
    }
    return schedule.get(station, {}).get(departure_time, "No train found")

### Prompting Mistral Large with Tool Use

Now, let's prompt Mistral Large to use the defined tool to provide the train schedule.

In [ ]:
# Function to prompt Mistral model
def prompt_mistral(prompt):
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "toolConfig": toolConfig,  # Provide Mistral with details about our tool
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    }

    response = bedrock_client.converse(**converse_api_params)
    
    if response['output']['message']['content'][0].get('toolUse'):
        tool_use = response['output']['message']['content'][0]['toolUse']
        tool_name = tool_use['name']
        tool_inputs = tool_use['input']

        if tool_name == "shinkansen_schedule":
            print("Mistral wants to use the shinkansen_schedule tool")
            station = tool_inputs["station"]
            departure_time = tool_inputs["departure_time"]
            
            try:
                result = shinkansen_schedule(station, departure_time)
                print("Train schedule result:", result)
            except ValueError as e:
                print(f"Error: {str(e)}")

    else:
        print("Mistral responded with:")
        print(response['output']['message']['content'][0]['text'])

In [ ]:
prompt_mistral("What train departs Tokyo at 9:00?")

## Adding another tool

### Weather Forecast Function

This function simulates fetching weather forecast data for a given city and date. The data is hardcoded for demonstration purposes.

- **Input**: City name and date (in YYYY-MM-DD format)
- **Output**: Weather forecast for the specified city and date

In [ ]:
def weather_forecast(city, date):
    # This is a mock function to simulate fetching weather forecast data
    forecast = {
        "Tokyo": {"2023-06-12": "Sunny", "2023-06-13": "Rainy"},
        "Osaka": {"2023-06-12": "Cloudy", "2023-06-13": "Sunny"}
    }
    return forecast.get(city, {}).get(date, "No forecast found")

## Define Tool Configuration

### Tool Configuration

This section defines the configuration for the tools used by the Mistral model. It includes the Shinkansen schedule tool and the weather forecast tool.

- **Shinkansen Schedule Tool**: Fetches train departure times for a specified station and time.
- **Weather Forecast Tool**: Fetches the weather forecast for a specified city and date.


In [ ]:
# Define the tool configuration
toolConfig = {
    "tools": [
        {
            "toolSpec": {
                "name": "shinkansen_schedule",
                "description": "Fetches Shinkansen train schedule departure times for a specified station and time.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "station": {
                                "type": "string",
                                "description": "The station name."
                            },
                            "departure_time": {
                                "type": "string",
                                "description": "The departure time in HH:MM format."
                            }
                        },
                        "required": ["station", "departure_time"]
                    }
                }
            }
        },
        {
            "toolSpec": {
                "name": "weather_forecast",
                "description": "Fetches the weather forecast for a specified city and date.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "city": {
                                "type": "string",
                                "description": "The city name."
                            },
                            "date": {
                                "type": "string",
                                "description": "The date in YYYY-MM-DD format."
                            }
                        },
                        "required": ["city", "date"]
                    }
                }
            }
        }
    ]
}

### Mistral Prompt Function

The `prompt_mistral` function sends a prompt to the Mistral model, handles the response, and uses the appropriate tool based on the model's output.

- **Input**: Prompt text for the Mistral model.
- **Process**:
  1. Sends the prompt to the Mistral model.
  2. Checks if the model wants to use a tool.
  3. If a tool is specified, fetches the necessary input data.
  4. Calls the appropriate function (`shinkansen_schedule` or `weather_forecast`) based on the tool specified.
  5. Outputs the result from the function or the model's text response.
- **Error Handling**: Catches and prints exceptions if any occur during the process.


### Tool Use Handling

This section of the code demonstrates how to handle different tool usage scenarios based on the tool name provided in the `tool_use` dictionary. It dynamically processes the tool inputs and calls the corresponding function to retrieve results. The supported tools include:

1. **Shinkansen Schedule**: Fetches the train schedule based on the specified station and departure time.
2. **Weather Forecast**: Retrieves the weather forecast for a given city and date.

Each tool's inputs are extracted and passed to their respective functions (`shinkansen_schedule` and `weather_forecast`). The results are printed, and any errors encountered during execution are caught and displayed.


In [ ]:
def prompt_mistral(prompt):
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "toolConfig": toolConfig,  # Provide Mistral with details about our tools
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    }
    
    try:
        response = bedrock_client.converse(**converse_api_params)
        
        tool_use = response['output']['message']['content'][0].get('toolUse')
        if tool_use:
            tool_name = tool_use['name']
            tool_inputs = tool_use['input']

            if tool_name == "shinkansen_schedule":
                station = tool_inputs["station"]
                departure_time = tool_inputs["departure_time"]
                
                try:
                    result = shinkansen_schedule(station, departure_time)
                    print("Train schedule result:", result)
                except ValueError as e:
                    print(f"Error: {str(e)}")
            
            elif tool_name == "weather_forecast":
                city = tool_inputs["city"]
                date = tool_inputs["date"]
                
                try:
                    result = weather_forecast(city, date)
                    print("Weather forecast result:", result)
                except ValueError as e:
                    print(f"Error: {str(e)}")

        else:
            print(response['output']['message']['content'][0]['text'])
    
    except Exception as e:
        print(f"Error occurred: {str(e)}")


In [ ]:
prompt_mistral("What is the weather forecast for Tokyo on 2023-06-12?")

## ToolChoice Configuration

Next, let's explore how different tool choices affect the model's behavior and the effectiveness of leveraging our train schedule tool. ToolChoice is defined within the toolConfig JSON object and cannot be passed as a separate argument.

### ToolChoice Options
Mistral Large supports two tool_choice options:
- **any**: The model must request at least one tool, and no text is generated unless a tool is used.
- **auto**: The model automatically decides whether to call a tool or generate text. This is the default option.

Currently, the **tool** option is not supported by Mistral Large.

### Setting ToolChoice to "any"

In this example, we'll configure the ToolChoice parameter to "any" and observe how Mistral Large responds. We'll use the same tool_config as before.

In [ ]:
tool_config = {
    "toolChoice": {"any": {}},
    "tools": [
        {
            "toolSpec": {
                "name": "shinkansen_schedule",
                "description": "Fetches Shinkansen train schedule departure times for a specified station and time.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "station": {
                                "type": "string",
                                "description": "The station name."
                            },
                            "departure_time": {
                                "type": "string",
                                "description": "The departure time in HH:MM format."
                            }
                        },
                        "required": ["station", "departure_time"]
                    }
                }
            }
        }
    ]
}

In [ ]:
def prompt_mistral_with_tool_choice(prompt):
    # Prepare the messages as a list of dictionaries
    messages = [{"role": "user", "content": [{"text": prompt}]}]  # Adjusted to match the expected structure

    # Prepare the API parameters
    converse_api_params = {
        "modelId": model_id,
        "messages": messages,
        "toolConfig": tool_config,  # Include the tool config here
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    }

    try:
        # Call the Bedrock API
        response = bedrock_client.converse(**converse_api_params)

        # Extract Mistral's response
        mistral_response = response['output']['message']['content'][0]['text']
        print("Mistral's response:")
        print(mistral_response)

    except Exception as e:
        print(f"An error occurred: {e}")

We'll ask a question that does not have to do with our train schedules in Tokyo, but a different question about trains.

In [ ]:
prompt_mistral_with_tool_choice("What is the bullet train in France called?")

In this example, Mistral Large correctly identifies that it does not have a tool for the requested information but still provides a useful response based on its internal knowledge.

## Conclusion

By following this guide, you should now have a solid understanding of how to leverage the Converse API for tool-use with the Mistral Large model. Experiment with different tools and configurations to fully explore the capabilities of Mistral-Large and enhance your AI-driven applications.